# 3.0 Combine data

In [18]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import math
from sklearn.preprocessing import Imputer
from sklearn.base import TransformerMixin
from sklearn import cross_validation
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.cross_validation import cross_val_score
import os

def sound():
    os.system('afplay /System/Library/Sounds/Sosumi.aiff')

%matplotlib inline
sns.set_context('notebook')

path = '/'.join(os.getcwd().split('/')[:-1])
print(path)

/Volumes/Dropbox/largetransfer/luc/carter


In [19]:
df0 = pd.read_csv(path+'/data/model_data/tci_2_0.csv', dtype={'Parcel ID':str}, index_col='parcel')
df1 = pd.read_csv(path+'/data/model_data/tci_2_1.csv', dtype={'Parcel ID':str}, index_col='parcel')
df2 = pd.read_csv(path+'/data/model_data/tci_2_2.csv', dtype={'Parcel ID':str}, index_col='parcel')
df3 = pd.read_csv(path+'/data/model_data/tci_2_3.csv', dtype={'Parcel ID':str}, index_col='parcel')

In [20]:
df = pd.merge(df0, df1[[col for col in df1.columns if col not in ['Date','vacant']]], how='inner', left_index=True, right_index=True)
df = pd.merge(df, df2[[col for col in df2.columns if col not in ['Date','vacant']]], how='inner', left_index=True, right_index=True)
df = pd.merge(df, df3[[col for col in df3.columns if col != 'Date']], how='inner', left_index=True, right_index=True)

In [21]:
df.columns

Index(['vacant', 'Date', 'Survey Category', 'Survey Grade', 'SPA_NAME',
       'cqual_num', 'occup', 'rnumstor', 'propsize', 'totusabl', 'tmktval',
       'condition13', 'condition14', 'condition_value13', 'condition_value14',
       'style_filtered', 'ownerocc_value', 'LATITUDE', 'LONGITUDE', 'totbldgs',
       'yrbuilt_filtered', 'rextwall', 'vacant_block', 'NAME10', 'median_rent',
       'property_crimes', 'burglaries', 'part_one_crimes', 'part_two_crimes',
       'bachelors+', 'poverty_rate', 'median_hh_income', 'white', 'black',
       'asian', 'other', 'hispanic', 'young', 'middle', 'old', 'lb_acquired',
       'lb_tax_fc', 'TOTAL_NET_DELQ_BALANCE', 'TAX_ASSESSED_LAND',
       'TAX_MARKET_LAND', 'LENDER_PROCESS_TYPE', 'GRAND_TOTAL_BALANCE',
       'GRAND_TOTAL_PAID', 'delq_total_ratio', 'paid_percent',
       'GRAND_TOTAL_BALANCE_14', 'pv_count', 'v_total_1yr', 'v_total_2yr',
       'days_since_ovv', 'c_ovv_num', 'days_since_ovv_board_up',
       'c_ovv_board_num', 'active_fc', '

In [22]:
df.loc[df.occup.notnull(),'occup'] = df[df.occup.notnull()].occup.apply(lambda x: int(x[0]))

In [23]:
df.loc[df.occup.isnull(),'occup'] = 1

In [24]:
tem = df[['SPA_NAME','totusabl']].groupby('SPA_NAME').agg(np.median)
tem = dict(tem['totusabl'])
df.loc[df.totusabl==0, 'totusabl'] = df[df.totusabl==0].apply(lambda x: tem[x.SPA_NAME], axis=1)

In [25]:
df['price_per_sqft'] = df['tmktval']/df['totusabl']

In [26]:
# tem = df[['SPA_NAME','tmktval','condition_value14','GRAND_TOTAL_BALANCE','sqft']].groupby('SPA_NAME').agg([np.median,np.mean])
# tem

In [27]:
# val = dict(tem['condition_value14']['mean'])
# df['cond_diff'] = df.apply(lambda x: x.condition_value14 - val[x.SPA_NAME], axis=1)

In [28]:
# val = dict(tem['tmktval']['median'])
# df['tmktval_diff'] = df.apply(lambda x: x.tmktval - val[x.SPA_NAME], axis=1)

In [29]:
# val = dict(tem['GRAND_TOTAL_BALANCE']['median'])
# df['balance_diff'] = df.apply(lambda x: x.GRAND_TOTAL_BALANCE - val[x.SPA_NAME], axis=1)

In [30]:
# k = 0.00005
# b = np.linspace(0,20000,20)
# sns.distplot(df.loc[df.vacant==1,'GRAND_TOTAL_BALANCE'], bins=b,kde_kws={'bw':k})
# sns.distplot(df.loc[df.vacant==0,'GRAND_TOTAL_BALANCE'], bins=b,kde_kws={'bw':k})
# plt.axis([min(b),max(b),0,.0009]);

In [31]:
# k = 0.00005
# b = np.linspace(-500,20000,20)
# sns.distplot(df.loc[df.vacant==1,'balance_diff'], bins=b,kde_kws={'bw':k})
# sns.distplot(df.loc[df.vacant==0,'balance_diff'], bins=b,kde_kws={'bw':k})
# plt.axis([min(b),max(b),0,.0009]);

In [32]:
# b = np.linspace(0,150000,40)
# sns.distplot(df.loc[df.vacant==1,'tmktval'], bins=b)
# sns.distplot(df.loc[df.vacant==0,'tmktval'], bins=b)
# plt.axis([0,max(b),0,0.000025]);

In [33]:
# sns.distplot(df.loc[df.vacant==1,'sqft'], bins=np.linspace(0,160,40))
# sns.distplot(df.loc[df.vacant==0,'sqft'], bins=np.linspace(0,160,40))
# plt.axis([0,150,0,0.035]);

In [34]:
# k = 0.5
# b = np.linspace(0,10,11)
# sns.distplot(df.loc[df.vacant==1,'condition_value14'], bins=b,kde_kws={'bw':k})
# sns.distplot(df.loc[df.vacant==0,'condition_value14'], bins=b,kde_kws={'bw':k})
# plt.axis([0,10,0,.9]);

In [35]:
# k = 3000
# b = np.linspace(-50000,50000,50)
# sns.distplot(df.loc[df.vacant==1,'tmktval_diff'], bins=b,kde_kws={'bw':k})
# sns.distplot(df.loc[df.vacant==0,'tmktval_diff'], bins=b,kde_kws={'bw':k})
# plt.axis([-50000,50000,0,.00005]);

In [36]:
# k = .3
# b = np.linspace(-5,5,20)
# sns.distplot(df.loc[df.vacant==1,'cond_diff'], bins=b,kde_kws={'bw':k})
# sns.distplot(df.loc[df.vacant==0,'cond_diff'], bins=b,kde_kws={'bw':k})
# plt.axis([-5,5,0,.9]);

In [37]:
# val = dict(tem['tmktval']['median'])
# df['mktval_diff'] = df.apply(lambda x: x.tmktval - val[x.SPA_NAME], axis=1)

In [38]:
# cols_temp = df.columns[:-4]

In [39]:
# df['missing_values'] = df.apply(lambda x: len(x[x.isnull()]),axis=1)
to_remove = ['Parcel ID','parcel','Date','Survey Category','Survey Grade',\
             'condition13', 'condition14', 'al_date','SPA_NAME','NAME10','style_filtered','rextwall','LENDER_PROCESS_TYPE']

In [40]:
features = list(df.columns.values)
for i in to_remove:
    if i in features:
        features.remove(i)

In [41]:
for i,j in enumerate(features):
    print(i,j)

0 vacant
1 cqual_num
2 occup
3 rnumstor
4 propsize
5 totusabl
6 tmktval
7 condition_value13
8 condition_value14
9 ownerocc_value
10 LATITUDE
11 LONGITUDE
12 totbldgs
13 yrbuilt_filtered
14 vacant_block
15 median_rent
16 property_crimes
17 burglaries
18 part_one_crimes
19 part_two_crimes
20 bachelors+
21 poverty_rate
22 median_hh_income
23 white
24 black
25 asian
26 other
27 hispanic
28 young
29 middle
30 old
31 lb_acquired
32 lb_tax_fc
33 TOTAL_NET_DELQ_BALANCE
34 TAX_ASSESSED_LAND
35 TAX_MARKET_LAND
36 GRAND_TOTAL_BALANCE
37 GRAND_TOTAL_PAID
38 delq_total_ratio
39 paid_percent
40 GRAND_TOTAL_BALANCE_14
41 pv_count
42 v_total_1yr
43 v_total_2yr
44 days_since_ovv
45 c_ovv_num
46 days_since_ovv_board_up
47 c_ovv_board_num
48 active_fc
49 fc_1yr
50 fc_2yr
51 fc_days
52 sa_days
53 t_days
54 t_shf_deed
55 t_num
56 al_num
57 al_days
58 price_per_sqft


In [42]:
# from sklearn import tree

# clf = tree.DecisionTreeClassifier(max_depth=2,min_samples_leaf=5)
# # iris = load_iris()
# clf = clf.fit(df3[features], df3.vacant)
# tree.export_graphviz(clf, out_file=path+'/tree.dot',feature_names=features, impurity=False, label=None,filled=True,class_names=['Not vacant','Vacant']) 

In [43]:
print(df[features].columns[df[features].dtypes == 'object'])

Index(['occup'], dtype='object')


In [44]:
# transform categorical variables (including NaN's)
# objs = df[features].columns[df[features].dtypes == 'object']
# prefixes = ['style','ext','lender']

# objs = ['LENDER_PROCESS_TYPE']
# prefixes = ['lender']

# df2 = df[objs]
# df3 = pd.concat([pd.get_dummies(df2[col], prefix=prefixes[index]) for index, col in enumerate(df2)], axis=1)

# for i in objs:
#     if i in features:
#         features.remove(i)
df3 = pd.DataFrame()
df3[features] = df[features]
df3 = df3.fillna(0)

print(df3.shape)

(113132, 59)


In [48]:
df3.to_csv(path+'/data/model_data/tci_final.csv')